# When Explanations Lie: Why Modified BP Attribution fails

This notebook produces the cosine similaries of the relevance vectors.

In [ ]:
# uncomment to install install packages
# !pip install tensorflow-gpu==1.13.1
# !pip install innvestigate seaborn tqdm

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf
import warnings

import innvestigate
import matplotlib.pyplot as plt

import numpy as np
import PIL 
import copy
import contextlib

import imp
import numpy as np
import os

from skimage.measure import compare_ssim 
import pickle
from collections import OrderedDict
from IPython.display import IFrame, display

import keras
import keras.backend
import keras.models


import innvestigate
import innvestigate.applications.imagenet
import innvestigate.utils as iutils
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.analyzer.relevance_based.relevance_analyzer import LRP
from innvestigate.analyzer.base import AnalyzerNetworkBase, ReverseAnalyzerBase
from innvestigate.analyzer.deeptaylor import DeepTaylor

import time
import tqdm

import seaborn as sns

import itertools
import matplotlib as mpl
from when_explanations_lie import *
from monkey_patch import custom_add_bn_rule
from tensorflow.python.client import device_lib

import deeplift

In [ ]:
device_lib.list_local_devices()

In [ ]:
# path to imagenet validation
imagenet_val_dir = "/mnt/ssd/data/imagenet/imagenet-raw/validation"
#imagenet_val_dir = "/home/leonsixt/tmp/imagenet/imagenet-raw/validation/"
# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
#n_selected_imgs = 200
n_selected_imgs = 10

load_weights = True
model_names = ['resnet50', 'vgg16']

In [ ]:
#keras.backend.clear_session()
#model, innv_net, color_conversion = load_model('vgg16', load_weights)
#ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
#    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)
#
#keras.backend.clear_session()
#model, innv_net, color_conversion = load_model('resnet50', load_weights)
#ex_image, ex_target, val_images, selected_img_idxs = load_val_images(
#    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)
#
#
#assert ((ex_image - ex_image_vgg) == 0).all()
#
#nice_layer_names = get_nice_layer_names(model)

# DeepLift

In [ ]:
keras.backend.clear_session()
#model, innv_net, color_conversion = load_model('vgg16', load_weights=True)
#ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
#    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)
#
#
#model_random, innv_net, color_conversion = load_model('vgg16', load_weights=False)
#ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
#    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

model_cascading, innv_net, color_conversion = load_model('resnet50', load_weights=True)
ex_image, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

In [ ]:
from deeplift.layers import Dense, Concat, Merge
from deeplift.layers.core import DenseMxtsMode, SingleInputMixin, Node, NoOp
from deeplift.conversion.kerasapi_conversion import KerasKeys
import keras.backend as K
from keras.utils.generic_utils import transpose_shape
import tensorflow as tf

In [ ]:
def monkey_patch_Merge_compute_shape(self, input_shapes):                       
    shape = []                                                                  
    # don't recompute input shapes
    # input_shapes = [an_input.get_shape() for an_input in self.inputs]         
    assert len(set(len(x) for x in input_shapes)) == 1, (
      "all inputs should have the same num"+                                    
      " of dims - got: "+str(input_shapes))          
    for dim_idx in range(len(input_shapes[0])):    
        lengths_for_that_dim = [input_shape[dim_idx]
                                for input_shape in input_shapes]
        if (dim_idx != self.axis):
            assert len(set(lengths_for_that_dim))==1, (
                   "lengths for dim "+str(dim_idx)  
                   +" should be the same, got: "+str(lengths_for_that_dim))
            shape.append(lengths_for_that_dim[0])
        else:
            shape.append(self.compute_shape_for_merge_axis(                     
                               lengths_for_that_dim))                           
    return shape     
Merge._compute_shape = monkey_patch_Merge_compute_shape

In [ ]:
# copied from: https://github.com/kundajelab/deeplift/pull/76/files

class GlobalAvgPool2D(SingleInputMixin, Node):

    def __init__(self, **kwargs):
        super(GlobalAvgPool2D, self).__init__(**kwargs)

    def _compute_shape(self, input_shape):
        assert len(input_shape)==4
        shape_to_return = [None, input_shape[-1]]
        return shape_to_return

    def _build_activation_vars(self, input_act_vars):
        return tf.reduce_mean(input_act_vars, axis=(1,2))

    def _build_pos_and_neg_contribs(self):
        inp_pos_contribs, inp_neg_contribs =\
            self._get_input_pos_and_neg_contribs()
        pos_contribs = self._build_activation_vars(inp_pos_contribs)
        neg_contribs = self._build_activation_vars(inp_neg_contribs)
        return pos_contribs, neg_contribs

    def _grad_op(self, out_grad):
        height = self._get_input_activation_vars().get_shape().as_list()[1]
        width = self._get_input_activation_vars().get_shape().as_list()[2]
        mask = (tf.ones_like(self._get_input_activation_vars())/
                float(width*height))
        return tf.multiply(out_grad[:, None, None, :], mask)

    def _get_mxts_increments_for_inputs(self):
        pos_mxts_increments = self._grad_op(self.get_pos_mxts())
        neg_mxts_increments = self._grad_op(self.get_neg_mxts())
        return pos_mxts_increments, neg_mxts_increments

    
def globalavgpooling2d_conversion(config, name, verbose, **kwargs):
    return [layers.GlobalAvgPool2D(name=name, verbose=verbose)]

    
class ZeroPadding2D(NoOp):
    def __init__(self, padding, data_format=None, **kwargs):                
        # self.rank is 1 for ZeroPadding1D, 2 for ZeroPadding2D.            
        self.rank = len(padding)                                            
        self.padding = padding                                              
        self.data_format = K.normalize_data_format(data_format)             
        super(ZeroPadding2D, self).__init__(**kwargs)                       
                                                                            
    def _compute_shape(self, input_shape):                                  
        padding_all_dims = ((0, 0),) + tuple(self.padding) + ((0, 0),)
        spatial_axes = list(range(1, 1 + self.rank))           
        padding_all_dims = transpose_shape(padding_all_dims,    
                                           self.data_format,    
                                           spatial_axes)        
        output_shape = list(input_shape)                       
        for dim in range(len(output_shape)):                    
            if output_shape[dim] is not None:                   
                output_shape[dim] += sum(padding_all_dims[dim]) 
        print("ZeroPadding", input_shape, output_shape, self.padding)
        return tuple(output_shape)                                          
                                                                            
    def _build_activation_vars(self, inputs):                               
        return K.spatial_2d_padding(inputs,                                 
                                    padding=self.padding,                   
                                    data_format=self.data_format) 

    def _build_pos_and_neg_contribs(self):
        input_pos_contribs, input_neg_contribs = self._get_input_pos_and_neg_contribs()
        return (self._build_activation_vars(input_pos_contribs), 
                self._build_activation_vars(input_neg_contribs))
        
    def _grad_op(self, out_grad):
        b, h, w, c = self._get_input_activation_vars().get_shape().as_list()
        print("ZeroPadding2D", self.padding)
        (ha, hb), (wa, wb) = self.padding
        return out_grad[:, ha:ha+h, wa:wa+w]
    
    def _get_mxts_increments_for_inputs(self):
        pos_mxts_increments = self._grad_op(self.get_pos_mxts())
        neg_mxts_increments = self._grad_op(self.get_neg_mxts())
        return pos_mxts_increments, neg_mxts_increments

    
def globalavgpooling2d_conversion(config, name, verbose, **kwargs):
    return [GlobalAvgPool2D(
             name=name,
             verbose=verbose)]


def zeropadding2d_conversion(config, name, verbose, **kwargs):                         
    #print(config, name, verbose, kwargs)                                              
    padding = config[KerasKeys.padding]                                                
    data_format = config[KerasKeys.data_format]                                        
    return [ZeroPadding2D(padding, data_format)]  



In [ ]:

class Add(Dense):                                                  
    """                                                            
    Is like a linear layer: [1 1] * [a b]^T                        
    """                                                            
    def __init__(self, **kwargs):         
        super(Add, self).__init__(                                 
            kernel=np.ones((2, 1)),                               
            bias=np.zeros((1,)),                                   
            dense_mxts_mode=DenseMxtsMode.Linear, **kwargs)        

        
class ConcatForAdd(Concat):
    def __init__(self, **kwargs):
        super().__init__(axis=4, **kwargs)
        
    def _build_activation_vars(self, input_act_vars):
        return super()._build_activation_vars([v[:, :, :, :, None] for v in input_act_vars])
 
    def _compute_shape(self, input_shape):
        input_shape_pad = [list(inp) + [1,] for inp in input_shape]
        output_shape = super()._compute_shape(input_shape_pad)
        return output_shape
    #def _build_pos_and_neg_contribs(self):
    #    pos, neg = super()._build_pos_and_neg_contribs()
    #    return [p[:, :, :, :, 0] for p in pos], [n[:, :, :, :, 0] for n in neg]
    
    def _get_mxts_increments_for_inputs(self):
        print("ConcatForAdd: MXTS inc")
        pos, neg = super()._get_mxts_increments_for_inputs()
        print(pos.shape, neg.shape)
        return [p[:, :, :, :, 0] for p in pos], [n[:, :, :, :, 0] for n in neg]

    def _get_mxts_increments_for_inputs(self):                                  
        return (
            [self.get_pos_mxts()[:, :, :, :, i] for i in range(len(self.inputs))],
            [self.get_neg_mxts()[:, :, :, :, i] for i in range(len(self.inputs))],
        )
    
    
class FlattenForAdd(NoOp):
    def _compute_shape(self, input_shape):                         
        b, h, w, c, n = input_shape             
        if b is None:
            warnings.warn("Assume batch_size = 1")
            b = 1
        self.shape_tuple = (None, h, w, c)
        return (None, n)                                     
    
    def _build_activation_vars(self, input_act_vars):              
        shape = input_act_vars.shape                               
        self.shape_tensor = tf.shape(input_act_vars)
        return tf.reshape(input_act_vars, (-1, shape[-1]))

    def _get_mxts_increments_for_inputs(self):              
        input_shape = tf.shape(self.inputs._activation_vars)
        print(input_shape)
        pos_mxts_increments = tf.reshape(self.get_pos_mxts(), input_shape)
        neg_mxts_increments = tf.reshape(self.get_neg_mxts(), input_shape)             
        return pos_mxts_increments, neg_mxts_increments       
    
class BackToConvForAdd(NoOp):
    def __init__(self, corresponding_flatten, **kwargs):
        self.corresponding_flatten = corresponding_flatten
        super().__init__(**kwargs)
        
    def _compute_shape(self, input_shape):
        return self.corresponding_flatten.shape_tuple
    
    def _build_activation_vars(self, input_act_vars):
        print("BackToConvForAdd", self.corresponding_flatten.shape_tensor)
        out = tf.reshape(input_act_vars, self.corresponding_flatten.shape_tensor[:-1])
        return out
    
    def _build_pos_and_neg_contribs(self):
        inp_pos_contribs, inp_neg_contribs = self._get_input_pos_and_neg_contribs()                  
        pos_contribs = self._build_activation_vars(inp_pos_contribs)
        neg_contribs = self._build_activation_vars(inp_neg_contribs)
        return pos_contribs, neg_contribs                           
                                                                    
    def _unflatten_keeping_first(self, mxts):                       
        input_act_vars = self._get_input_activation_vars()          
        print("BackToConvForAdd _unflatten", tf.shape(input_act_vars))
        return tf.reshape(tensor=mxts,                              
                          shape=tf.shape(input_act_vars))           
                                                                    
    def _get_mxts_increments_for_inputs(self):              
        input_shape = tf.shape(self.inputs._activation_vars)
        print(input_shape)
        pos_mxts_increments = tf.reshape(self.get_pos_mxts(), (-1, 1))
        neg_mxts_increments = tf.reshape(self.get_neg_mxts(), (-1, 1))             
        return pos_mxts_increments, neg_mxts_increments       
        
        
def add_conversion(config, name, verbose, **kwargs):               
    #print(config, name, verbose, kwargs)    
    flatten = FlattenForAdd() 
    return deeplift.util.connect_list_of_layers([ConcatForAdd(), flatten, Add(), BackToConvForAdd(flatten)])
    #return deeplift.util.connect_list_of_layers([ConcatForAdd(), flatten, Add(), BackToConvForAdd(flatten)])

In [ ]:

from deeplift.conversion.kerasapi_conversion import layer_name_to_conversion_function as deeplift_layer_name_to_conversion_function

if 'copy_layer_name_to_conversion_function' not in globals():
    copy_layer_name_to_conversion_function = copy.deepcopy(deeplift_layer_name_to_conversion_function)
    
def monkey_patched_layer_name_to_conversion_function(layer_name):  
    try: 
        return copy_layer_name_to_conversion_function(layer_name)
    except KeyError:
        return {
           'zeropadding2d': zeropadding2d_conversion,                                                                                                                                                                                                                              
           'globalaveragepooling2d': globalavgpooling2d_conversion,                                                                                                                                                                                                                              
           'add': add_conversion,                 
       }[layer_name.lower()]
    
deeplift.conversion.kerasapi_conversion.layer_name_to_conversion_function = \
    monkey_patched_layer_name_to_conversion_function

In [ ]:
nonlinear_mode  = 'rescale'  # or reveal_cancel
nonlinear_mode  = 'reveal_cancel'

dl_original = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode=nonlinear_mode)


In [ ]:
%pdb off
hmap_original = dl_original.analyze(ex_image)

In [ ]:
dp_model = dl_original._deeplift_model
dp_layers = dp_model._name_to_layer
dp_layer_names = list(dp_layers.keys())

In [ ]:
dp_layer_names

In [ ]:
dir(dp_model)

In [ ]:
def deeplift_prediction(dp_model, output_layer=None):
    inp_layer_name = dp_model._input_layer_names[0]
    inp_layer = dp_model._name_to_layer[inp_layer_name]
    if output_layer is None:
        output_layer = list(dp_model._name_to_layer.values())[-1]
    
    outputs = output_layer.get_activation_vars()
    def wrapper(inp_val, ref_val):
        sess = K.get_session()
        return sess.run([outputs], feed_dict={
            inp_layer.get_activation_vars(): inp_val,
            inp_layer.get_reference_vars(): ref_val,
        })[0]
    return wrapper

In [ ]:
dp_predition_func = deeplift_prediction(dp_model)
out_deeplift = dp_predition_func(ex_image, np.zeros_like(ex_image))

In [ ]:
out_deeplift

In [ ]:
out_keras = model_cascading.predict(ex_image)
np.abs(out_deeplift - out_keras).mean() 

In [ ]:
np.abs(out_deeplift - out_keras).max() 

In [ ]:
dl_original._deeplift_model

In [ ]:
plt.imshow(hmap_original[0].sum(-1))

In [ ]:
conv2_1 = list(dp_layers.values())[6]
input_layer = list(dp_layers.values())[0]
target_layer = list(dp_layers.values())[-1]
target_layer

In [ ]:
input_layer.get_activation_vars(), input_layer.get_reference_vars()

In [ ]:

input_layer._diff_from_reference_vars

In [ ]:
conv2_1._get_input_diff_from_reference_vars() == list(dp_layers.values())[5]._diff_from_reference_vars

In [ ]:

conv2_1._diff_from_reference_vars

In [ ]:
# how to handle different layer idxs?

In [ ]:
class DeepLiftCSC:
    def __init__(self, deeplift_wrapper):
        self.deeplift_wrapper = deeplift_wrapper
        self.model = self.deeplift_wrapper._deeplift_model
        self.layers = list(self.model._name_to_layer.values())
        self.layer_names = list(self.model._name_to_layer.keys())
        self.input_layer = self.layers[0]
        
    def get_relevances(self, input_value, set_layer_idx, relevance_value, reference=None):
        if reference is None:
            reference = np.zeros_like(input_value)
            
        self.layers[-1].set_active()
        changed_layer = self.layers[set_layer_idx]
        sess = keras.backend.get_session()
        contribs = sess.run(
            [layer._pos_mxts for layer in self.layers[:set_layer_idx]], 
            #[layer._pos_mxts for layer in self.layers[:set_layer_idx]], 
            feed_dict={
                self.input_layer.get_activation_vars(): ex_image,
                self.input_layer.get_reference_vars(): np.zeros_like(ex_image),
                changed_layer._pos_mxts: relevance_value,
                changed_layer._neg_mxts: relevance_value,
        })
        return contribs

    
class ModifyWeights:
    def __init__(self, model_trained, model_random, model):
        pass
    
    def copy_trained(self, layer_idxs=None):
        pass
    
    def copy_random(self, layer_idxs=None):
        pass


class DeepLiftModifyWeights(ModifyWeights):
    def __init__(self, model_trained, model_random):
        pass

In [ ]:
dl_csc = DeepLiftCSC(dl_original)

In [ ]:
len(dl_csc.layers)

In [ ]:
len(dl_csc.layers)

In [ ]:
len(model_cascading.layers)

In [ ]:
rows = 1
cols = 5

s = 5
fig, axes = plt.subplots(rows, cols, figsize=(s*cols, s*rows))

rel_multi = []
for i, ax in enumerate(axes.flatten()):
    n_sampels = 1
    rel_val = np.random.normal(size=(n_sampels, 1000)) 
    rel_val /= np.linalg.norm(rel_val)
    rel_per_layer = dl_csc.get_relevances(
        np.repeat(ex_image, n_sampels, axis=0),  set_layer_idx=159, 
        relevance_value=rel_val)
    
    im = ax.imshow(normalize(rel_per_layer[0][0].sum(-1)))
    plt.colorbar(im, ax=ax)
    rel_multi.append(rel_per_layer)

In [ ]:
def cosine_similarity_dot(U, V):
    v_norm =  V / np.linalg.norm(V, axis=1, keepdims=True)
    u_norm = U / np.linalg.norm(U, axis=1, keepdims=True)
    return (v_norm * u_norm).sum(1)

In [ ]:
a = np.random.uniform(size=(10, 5))
b = np.random.uniform(size=(10, 5))
cosine_similarity_dot(a, b).shape
cosine_similarity_dot(a, b),  cosine_similarity_dot(a, a)

In [ ]:
print("total relevance for 1 sample in MB:", sum([rel.size * 4 for rel in rel_per_layer]) / 10**6)

In [ ]:
%%timeit
for idx in range(0, 30):
    for i, j in itertools.combinations(range(len(rel_multi)), 2):
        b, h, w, c = rel_multi[i][idx].shape
        cos = cosine_similarity_dot(rel_multi[i][idx].reshape(b*h*w, c), rel_multi[j][idx].reshape(b*h*w, c))

In [ ]:
for idx in range(0, 30):
#idx = 30
    
    try:
        b, h, w, c = rel_multi[0][idx].shape
        print(rel_multi[0][idx].reshape(b*h*w, c).shape)
        cos = cosine_similarity_dot(rel_multi[0][idx].reshape(b*h*w, c), rel_multi[1][idx].reshape(b*h*w, c))
        print(cos.shape)
        _ = plt.hist(cos.flatten(), bins=40) #, range=[0.99, 1])
        plt.title(idx)
        plt.show()
    except ValueError:
        pass

In [ ]:
for idx in range(0, 30):
    try:
        b, h, w, c = rel_multi[0][idx].shape
        cos = cosine_similarity(rel_multi[0][idx].reshape(b*h*w, c), rel_multi[1][idx].reshape(b*h*w, c))
        _ = plt.hist(cos.flatten(), bins=40, range=[-1, 1])
        plt.title(idx)
        plt.show()
    except Exception:
        pass

In [ ]:
raise

In [ ]:
rel_multi[0][30] - rel_multi[1][30]

In [ ]:
(rel_per_layer[0] == 0).all()

In [ ]:
rel_per_layer[0].shape

In [ ]:
plt.imshow(rel_per_layer[1][0].sum(-1))
plt.colorbar()

In [ ]:

plt.imshow(rel_per_layer[0][0].sum(-1))

In [ ]:

sess = keras.backend.get_session()
dp_model._set_scoring_mode_for_target_layer(target_layer)
target_layer.set_active()

contrib = sess.run([input_layer.get_target_contrib_vars()], {
    input_layer.get_activation_vars(): ex_image,
    input_layer.get_reference_vars(): np.zeros_like(ex_image),
    conv2_1._get_input_diff_from_reference_vars(): 200*np.random.normal(size=(1, 112, 112, 64)),
    #conv2_1._pos_contribs: -50*np.ones(shape=(1, 112, 112, 128)),
    #conv2_1._neg_contribs: -50*np.ones(shape=(1, 112, 112, 128)),
    #conv2_1._pos_contribs: np.random.normal(size=(1, 112, 112, 128)),
    #conv2_1._neg_contribs: np.random.normal(size=(1, 112, 112, 128)),
    #conv2_1._diff_from_reference_vars: np.random.normal(size=(1, 112, 112, 128)),
})

In [ ]:
plt.imshow(contrib[0][0].sum(-1))
plt.colorbar()

In [ ]:
plt.imshow(contrib_a[0][0].sum(-1))
plt.colorbar()

In [ ]:
class ModelModifier:
    def __init__(self, model):
        self.model = model
        
    def copy_weights(self, other_model, layer_indices):
        copy_weights(self.model, other_model, layer_indices)
    
    def get_relevances(self, set_relevances_to=None, set_at_layer=None):
        pass
    
    def get_layer_idx(self, name):
        pass
    
    def get_layer_name(self, idx):
        pass
    
    def get_output_shape(self, layer_idx):
        pass

In [ ]:
np.abs(normalize(contrib_a[0]) - normalize(contrib[0])).mean()

In [ ]:
plt.imshow(contrib[0][0].sum(-1) - contrib_a[0][0].sum(-1))
plt.colorbar()

In [ ]:
raise

In [ ]:

list(dp_layers.keys())[6]

In [ ]:
func_all = dl_original._deeplift_model._get_func(
    find_scores_layer_name=[n for (n, _) in layers],
    pre_activation_target_layer_name=layers[-1][0],
    func_type=deeplift.models.FuncType.contribs,
)

In [ ]:
contribs_per_layer = func_all(input_data_list=[ex_image], batch_size=1, progress_update=1, task_idx=1)

In [ ]:
len(contribs_per_layer)

In [ ]:
plt.imshow(np.maximum(0, contribs_per_layer[9][0]).sum(-1))

In [ ]:
plt.imshow(np.minimum(0, contribs_per_layer[9][0]).sum(-1))

In [ ]:
for contrib in contribs_per_layer:
    print(contrib[0].shape)

In [ ]:

dl_original._deeplift_model._get_

In [ ]:
layers  = list(

In [ ]:
sess = K.backend.get_session()

sess.run([
    layer._pos_contribs, layer._neg_contribs
], feed_dict={
    
})



In [ ]:
%%timeit
hmap_original = dl_original.analyze(ex_image)

In [ ]:
dir(dl_original)

In [ ]:
raise

In [ ]:
# Sanity checks on one sample

In [ ]:
nonlinear_mode  = 'rescale'  # or reveal_cancel
copy_weights(model_cascading, model, range(0, original_idx))
dl_original = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode=nonlinear_mode)
out_original = model_cascading.predict(ex_image)
hmap_original = dl_original.analyze(ex_image)


copy_weights(model_cascading, model_random, range(layer_idx, original_idx))
out_random = model_cascading.predict(ex_image)

dl_random = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode=nonlinear_mode)
hmap_random = dl_random.analyze(ex_image)

In [ ]:
print('logit l1-mean:',  np.abs(out_original - out_random).mean())

plt.imshow(hmap_original[0].sum(-1))
plt.colorbar()
plt.show()
plt.imshow(hmap_random[0].sum(-1))
plt.colorbar()
plt.show()

plt.imshow(hmap_original[0].sum(-1) - hmap_random[0].sum(-1))
plt.colorbar()
plt.show()

In [ ]:
nonlinear_mode  = 'reveal_cancel'
copy_weights(model_cascading, model, range(0, original_idx))
dl_original = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode=nonlinear_mode)
out_original = model_cascading.predict(ex_image)
hmap_original = dl_original.analyze(ex_image)


copy_weights(model_cascading, model_random, range(layer_idx, original_idx))
out_random = model_cascading.predict(ex_image)

dl_random = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode=nonlinear_mode)
hmap_random = dl_random.analyze(ex_image)

In [ ]:
print('logit l1-mean:',  np.abs(out_original - out_random).mean())

plt.imshow(hmap_original[0].sum(-1))
plt.colorbar()
plt.show()
plt.imshow(hmap_random[0].sum(-1))
plt.colorbar()
plt.show()

plt.imshow(hmap_original[0].sum(-1) - hmap_random[0].sum(-1))
plt.colorbar()
plt.show()

In [ ]:
# nonlinear_mode  = rescale or reveal_cancel
copy_weights(model_cascading, model, range(0, original_idx))
dl_original = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode='reveal_cancel')


copy_weights(model_cascading, model_random, range(layer_idx, original_idx))
# nonlinear_mode  = rescale or reveal_cancel
dl_random = innvestigate.analyzer.DeepLIFTWrapper(model_cascading, nonlinear_mode='reveal_cancel')


hmap_original = dl_original.analyze(ex_image)
hmap_random = dl_random.analyze(ex_image)


plt.imshow(hmap_original[0].sum(-1))
plt.imshow(hmap_random[0].sum(-1))

In [ ]:
np.abs(out_original - out_random).mean()

In [ ]:

plt.imshow(hmap_original[0].sum(-1))
plt.show()
plt.imshow(hmap_random[0].sum(-1))
plt.show()

In [ ]:
hmap_original - hmap_random

In [ ]:
%%timeit 
hmap_random = dl_random.analyze(ex_image)

In [ ]:
%%timeit
hmap = dl.analyze(ex_image)

In [ ]:
self._func(task_idx=neuron_idx,
           input_data_list=X,
           batch_size=batch_size,
           input_references_list=self._references,
           progress_update=1000000)

In [ ]:
plt.imshow(hmap[0].sum(-1))
plt.colorbar()

In [ ]:
dl.de

In [ ]:
plt.imshow(hmap[0].sum(-1))
plt.colorbar()

In [ ]:
def parse_reversed(hidden):
    return [h[1] for h in hidden[1:]]


dead_neuron_mask = {}

for model_name in model_names:
    keras.backend.clear_session()
    model, innv_net, _ = load_model(model_name, load_weights=True)
    analyser = innvestigate.create_analyzer(
        "gradient", model, reverse_keep_tensors=True)
    
    analyser.analyze(np.concatenate([img for (img, _) in val_images[:20]], 0))
    
    grad_hidden = parse_reversed(analyser._reversed_tensors) 
    dead_neuron_mask[model_name] = [(0 == np.mean(g, 0, keepdims=True)).all(-1, keepdims=True) for g in grad_hidden]

In [ ]:
for model_name in model_names:
    plt.title(model_name + " - active neurons")
    plt.plot([(m.sum(-1) / m.shape[-1] > 0.999999).mean() for m in dead_neuron_mask[model_name]])
    plt.show()

In [ ]:
nice_layer_names

In [ ]:
histogram_layers = copy.deepcopy(replacement_layers)
histogram_layers['vgg16'].extend(['conv1_1', 'input'])
histogram_layers['resnet50'].extend(['conv2_1a', 'input'])

In [ ]:

histogram_layers_idx = OrderedDict()
for model_name in model_names:
    histogram_layers_idx[model_name] = []
    for layer_name in histogram_layers[model_name]:
        idx = get_layer_idx_full(model_name, nice_layer_names, layer_name)
        histogram_layers_idx[model_name].append(idx) 
histogram_layers_idx

In [ ]:
len(dead_neuron_mask['vgg16'])

In [ ]:
replacement_layers

In [ ]:
from innvestigate.analyzer.relevance_based.relevance_rule import AlphaBetaRule

def alpha_beta_wrapper(alpha, beta):
    class AlphaBetaRuleWrapper(AlphaBetaRule):
        def __init__(self, layer, state, bias=True, copy_weights=False):
            super(AlphaBetaRuleWrapper, self).__init__(layer, state, alpha=alpha, beta=beta, 
                             bias=bias, copy_weights=copy_weights)
            
        def __repr__(self):
            return "AlphaBetaRuleWrapper(alpha={}, beta={})".format(self._alpha, self._beta)
        
    return AlphaBetaRuleWrapper

def get_custom_rule(innv_name, kwargs):
    if innv_name == 'lrp.alpha_beta':
        return alpha_beta_wrapper(kwargs['alpha'], kwargs['beta'])
    elif innv_name == 'lrp.sequential_preset_a':
        return alpha_beta_wrapper(1, 0)
    elif innv_name == 'lrp.sequential_preset_b':
        return alpha_beta_wrapper(2, 1)
        
for label, innv_name, _, excludes, kwargs in analysers:
    print(innv_name, get_custom_rule(innv_name, kwargs))

In [ ]:
replacement_layers= {'vgg16': ['fc3'], 'resnet50': ['dense']}

In [ ]:
model_names, replacement_layers

In [ ]:
# replacement_layer_indices = [22]
n_sampled_v = 5

cos_sim_histograms = {}
cos_mean = {}
selected_percentiles = [0, 1, 5, 10, 20, 50, 100]
cos_sim_percentiles = {}

for label, innv_name, _, excludes, kwargs in tqdm.tqdm_notebook(analysers[4:]):
    if 'exclude_cos_sim' in excludes:
        continue
    for model_name in model_names[:1]:
        if 'exclude_' + model_name in excludes:
            continue
        keras.backend.clear_session()
        model_wo_softmax, innv_net, _ = load_model(model_name, load_weights=load_weights)
        if innv_name == "pattern.attribution":
            kwargs['patterns'] = innv_net['patterns']

        for replacement_layer in replacement_layers[model_name]:
            replacement_layer_idx = get_layer_idx_full(model_name, nice_layer_names, replacement_layer)
            custom_rule = get_custom_rule(innv_name, kwargs)
            with custom_add_bn_rule(custom_rule):
                repl_analyser, repl_shape = get_replacement_analyser(
                    model_wo_softmax, innv_name,  
                    replacement_layer_idx=replacement_layer_idx,
                    **kwargs)
                # repl_analyser.create_analyzer_model()
                cos_per_img = OrderedDict()
                for img_idx, (img, _) in tqdm.tqdm_notebook(zip(selected_img_idxs, val_images), 
                    desc="[{}.{}] {}".format(model_name, replacement_layer, label)):
                    channels = repl_shape[-1]
                    if label == "$\\alpha=100, \\beta=99$-LRP":
                        # a=100,b=99 sufferes numerical instabilities with std = 1
                        std = 1 / np.sqrt(channels)
                    else:
                        std = 1

                    relevance_v1 = std*np.random.normal(size=(1, ) + repl_shape[1:]) 
                    hmap = repl_analyser.analyze([img, relevance_v1])
                    intermediate_values = parse_reversed(repl_analyser._reversed_tensors)

                    relevance_v2 = std * np.random.normal(size=(n_sampled_v,) + repl_shape[1:]) 
                    img_tiled = np.tile(img, (n_sampled_v, 1, 1, 1))
                    outs = repl_analyser.get_cosine(img_tiled, relevance_v2,  intermediate_values[::-1])
                    outs = outs[::-1]
                    for layer_idx, (o, dead_neuron) in enumerate(zip(outs, dead_neuron_mask[model_name])):
                        cos_for_layer = np.abs(o)
                        # we filter 0 cosine similarites as they only appear practically when the gradients are zero
                        cos_per_img[model_name, layer_idx, img_idx] = cos_for_layer[cos_for_layer != 0]

                median_for_label = []
                percentile_for_label = OrderedDict([(p, []) for p in selected_percentiles])
                for layer_idx in range(n_layers[model_name]):
                    cos_per_layer = np.concatenate([cos_per_img[model_name, layer_idx, img_idx]  for img_idx in selected_img_idxs])
                    cos_per_layer = cos_per_layer.flatten()

                    idx = (label, model_name, replacement_layer_idx,  layer_idx)
                    cos_mean[idx] = np.mean(cos_per_layer)

                    perc_values = np.percentile(cos_per_layer,  selected_percentiles)
                    for p, val in zip(selected_percentiles, perc_values):
                        percentile_for_label[p].append(val)

                    if layer_idx in histogram_layers_idx[model_name]:

                        if len(cos_per_layer) > 50000:
                            ridx = np.random.choice(len(cos_per_layer), 50000, replace=False)
                            cos_per_layer_sel = cos_per_layer[ridx]
                        else:
                            cos_per_layer_sel = cos_per_layer

                        cos_sim_histograms[idx] = np.histogram(cos_per_layer_sel, bins)


                for p, values in percentile_for_label.items():
                    cos_sim_percentiles[label, model_name, replacement_layer_idx, p] = np.array(values)

In [ ]:
with keras.backend.get_session().as_default():
    print(1 - tf.losses.cosine_distance([0, 0], [0, 0], 0).eval())

In [ ]:
outs[0].shape, outs[-4].shape

In [ ]:
save_results = False
if save_results:
    os.makedirs('cache', exist_ok=True)
    with open('cache/cos_sim_with_hist_random_weights.pickle', 'wb') as f:
        pickle.dump((cos_sim_percentiles, cos_sim_histograms ), f)

In [ ]:
load_results = False
if load_results:
    os.makedirs('cache', exist_ok=True)
    with open('cache/cos_sim_with_hist.pickle', 'rb') as f:
        cos_sim_percentiles, cos_sim_histograms = pickle.load(f)

In [ ]:
def cosine_similarity(U, V):
    v_norm = V / np.linalg.norm(V, axis=0, keepdims=True)
    u_norm = U / np.linalg.norm(U, axis=0, keepdims=True)
    return v_norm.T @ u_norm

def get_sample_cos_sim_per_layer(output_shapes):
    values = []
    for layer_idx, shp in output_shapes.items():
        ch = shp[-1]
        n_samples = 1000
        u = np.random.normal(size=(ch, n_samples))
        v = np.random.normal(size=(ch, n_samples))
        cos = cosine_similarity(v, u)
        mask = np.tri(cos.shape[0])
        values.append(np.median(np.abs(cos[mask == 1])))
    return np.array(values)
        

In [ ]:
cos_sim_baseline = {}

for model_name in model_names:
    keras.backend.clear_session()
    model, _, _ = load_model(model_name)
    output_shapes = get_output_shapes(model)
    print(len(output_shapes))
    cos_sim_baseline[model_name] = get_sample_cos_sim_per_layer(output_shapes)

In [ ]:
model_names

In [ ]:
cos_sim_baseline['vgg16'].shape, cos_sim_baseline['resnet50'].shape

In [ ]:
legend = OrderedDict()

os.makedirs('figures/cosine_similarity', exist_ok=True)
for model_name in model_names[::-1]:
    for replacement_layer in replacement_layers[model_name]:
        repl_idx = get_layer_idx_full(model_name, nice_layer_names, replacement_layer)
        start_layer = n_layers[model_name] - repl_idx 
        
        layer_names = [name for idx, name in nice_layer_names[model_name].items()
                       if idx <= repl_idx][::-1]
        layer_idx = np.array([idx for idx, name in nice_layer_names[model_name].items()
                       if idx < repl_idx][::-1])
        
        print(layer_idx, repl_idx, start_layer)
        #layer_idx = layer_idxs
        
        plt.figure(figsize=(max(3, len(layer_idx) / 4), 3.5))
        
        for i, (label, _, _, _, _) in enumerate(analysers):
            idx = (label, model_name, repl_idx, 50)
            if idx not in cos_sim_percentiles:
                warnings.warn("not found: " + str(idx))
                continue
            print(len(cos_sim_percentiles[idx]))
            cos_sim_per_label = cos_sim_percentiles[idx][layer_idx]
            
            #cos_sim_per_label = []
            #for lidx in layer_idx:
            #    cos_sim_per_label.append(cos_mean[label, model_name, repl_idx, lidx])
            # try:
            #     cos_sim_per_label = cos_sim_percentiles[idx][layer_idx]
            # except IndexError:
            #     cos_sim_per_label = (cos_sim_baseline[model_name][layer_idx[:1]].tolist() +
            #                          cos_sim_percentiles[idx][layer_idx[1:]].tolist())
                
            plt.plot(0.5 + np.arange(len(cos_sim_per_label)), cos_sim_per_label, label=label, **mpl_styles[label])
            
            if label not in legend:
                legend[label] = mpl_styles[label]
            
        # Random Cos Similarity
        # Cos Similarity Base.
        label='Cos Similarity BL'
        style = {'color': (0.25, 0.25, 0.25)}
        plt.plot(0.5 + np.arange(len(layer_idx)), cos_sim_baseline[model_name][layer_idx], 
                 # label='Cos. Sim. Baseline', 
                 label=label,
                 **style)
        if label not in legend:
            legend[label] = style
        
        #plt.legend(bbox_to_anchor=(1, 1))
        plt.ylabel('cosine similarity')
        plt.xticks(np.arange(len(layer_names)), layer_names, rotation=90)
        plt.ylim(-0.05, 1.05)
        plt.grid('on', alpha=0.35) #, axis="y")
        plt.savefig("./figures/cosine_similarity/{}_layer_{}.pdf".format(model_name, repl_idx),  
                    bbox_inches='tight', pad_inches=0)
        plt.show()
        plt.close()

In [ ]:
list(cos_mean.keys())[0]

In [ ]:
plt.figure(figsize=(2.5, 3))
for label, style in legend.items():
    plt.plot([], label=label, alpha=1, **style)

plt.axis('off')
plt.legend(loc='center')
plt.savefig("./figures/cos_sim_legend.pdf",
            bbox_inches='tight', pad_inches=0.02)

In [ ]:
display(IFrame("./figures/cos_sim_legend.pdf", 800, 600))

In [ ]:
for attr_name, model_name, layer_idx, percentile in cos_sim_percentiles.keys():
    if attr_name == 'GuidedBP' and model_name == 'resnet50':
        print(attr_name, model_name, layer_idx, percentile)
    

In [ ]:
cos_sim_histograms.keys()

In [ ]:


attr_counts = []
labels = []
for (attr_name, model_name, repl_layer, layer_idx), (counts, bins) in cos_sim_histograms.items():
    if layer_idx != 7:
        #print(layer_idx)
        continue
    lower_09 = counts[bins[:-1] < 0.9].sum()
    print(attr_name, counts.sum())
    counts_collapsed = np.concatenate([lower_09[None], counts[bins[:-1] >= 0.9]])
    bins_int = np.arange(len(counts_collapsed) + 1)
    attr_counts.append(counts_collapsed)
    labels.append(attr_name)
plt.hist([bins_int[:-1]] * len(attr_counts), bins_int, 
         weights=attr_counts, stacked=True, label=labels)
plt.xticks(bins_int, ["{:.4g}".format(b) for b in [0] + bins[bins >= 0.9].tolist()], rotation=0)
plt.legend()

In [ ]:
bins

In [ ]:

hist[1]

In [ ]:
counts